In [1]:
import os
import glob
import pandas as pd

In [2]:
# Define working directory
os.chdir(r"C:\Users\hguzm\Documents\000. Personal\Bootcamp\Proyectos\FinalProject-Spotify\Original_data")

In [3]:
# Use glob to match the pattern ‘csv’
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

# Combine all files in the list and export as CSV
df = pd.concat([pd.read_csv(f) for f in all_filenames ])
df.reset_index(drop=True, inplace=True)
df.head(5)

,track,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,target
0,Lucky Man,Montgomery Gentry,spotify:track:4GiXBCUF7H6YfNQsnBRIzl,0.578,0.471,4,-7.270,1,0.0289,0.368000,0.00000,0.159,0.532,133.061,196707,4,30.88059,13,1
1,On The Hotline,Pretty Ricky,spotify:track:1zyqZONW985Cs4osz9wlsu,0.704,0.854,10,-5.477,0,0.1830,0.018500,0.00000,0.148,0.688,92.988,242587,4,41.51106,10,1
2,Clouds Of Dementia,Candlemass,spotify:track:6cHZf7RbxXCKwEkgAZT4mY,0.162,0.836,9,-3.009,1,0.0473,0.000111,0.00457,0.174,0.300,86.964,338893,4,65.32887,13,0
3,"Heavy Metal, Raise Hell!",Zwartketterij,spotify:track:2IjBPp2vMeX7LggzRN3iSX,0.188,0.994,4,-3.745,1,0.1660,0.000007,0.07840,0.192,0.333,148.440,255667,4,58.59528,9,0
4,I Got A Feelin',Billy Currington,spotify:track:1tF370eYXUcWwkIvaq3IGz,0.630,0.764,2,-4.353,1,0.0275,0.363000,0.00000,0.125,0.631,112.098,193760,4,22.62384,10,1


In [4]:
df2 = df.drop(['track', 'artist', 'uri'], axis=1)

In [5]:
# Reformat data
data = df2.values
X = data[:, 0:15]  
y = data[:, 15]

print(data.shape, X.shape, y.shape)
data
# X[0]
# y

(41106, 16) (41106, 15) (41106,)


array([[ 0.578  ,  0.471  ,  4.     , ..., 30.88059, 13.     ,  1.     ],
       [ 0.704  ,  0.854  , 10.     , ..., 41.51106, 10.     ,  1.     ],
       [ 0.162  ,  0.836  ,  9.     , ..., 65.32887, 13.     ,  0.     ],
       ...,
       [ 0.562  ,  0.314  , 10.     , ..., 21.11763, 10.     ,  1.     ],
       [ 0.622  ,  0.781  ,  7.     , ..., 47.13558, 11.     ,  0.     ],
       [ 0.664  ,  0.739  ,  2.     , ..., 42.50341, 14.     ,  1.     ]])

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

## Data Preprocessing

In [7]:
#Scale features (X)
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test) 

X_train_scaled

array([[ 1.35315003, -0.22396167,  1.07008741, ...,  0.24964134,
         0.64353579,  0.1078218 ],
       [-0.03430522,  0.9971179 ,  1.07008741, ...,  0.24964134,
         0.88344544, -0.71419692],
       [-0.2599077 ,  0.34296813,  1.07008741, ...,  0.24964134,
         0.72333873, -0.91970159],
       ...,
       [ 0.16873701, -0.838466  ,  1.07008741, ...,  0.24964134,
         0.07947695,  1.13534519],
       [-0.71111266, -1.65912662,  0.50456543, ...,  0.24964134,
         0.66731336, -0.50869224],
       [-1.39920022, -1.31024674,  1.07008741, ..., -2.13381886,
        -0.45146094,  0.1078218 ]])

One-hot encode the labels

In [8]:
#One-hot encode output labels (y)
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

y_train_categorical

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

## Creating and defining our Model Architecture

In [9]:
#First create a sequential model
from tensorflow.keras.models import Sequential

model = Sequential() 

In [10]:
#For categorical data, we use a classifier model (Softmax)
from tensorflow.keras.layers import Dense
number_inputs = 15  
number_hidden_nodes = 12

#Create hidden layer
model.add(Dense(units=number_hidden_nodes,activation='relu', input_dim=number_inputs))

#Create output layer
number_classes = 2
model.add(Dense(units=number_classes, activation='softmax')) 

Number of Hidden Nodes:
https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw#:~:text=The%20number%20of%20hidden%20neurons,size%20of%20the%20input%20layer.

In [11]:
#Model Summary
model.summary() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                192       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 26        
Total params: 218
Trainable params: 218
Non-trainable params: 0
_________________________________________________________________


In [12]:
#Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Training the Model

In [13]:
#Training the Model
model.fit(X_train_scaled,y_train_categorical,epochs=150, shuffle=True, verbose=2)

Epoch 1/150
964/964 - 1s - loss: 0.5610 - accuracy: 0.7100
Epoch 2/150
964/964 - 1s - loss: 0.5110 - accuracy: 0.7468
Epoch 3/150
964/964 - 1s - loss: 0.4980 - accuracy: 0.7555
Epoch 4/150
964/964 - 1s - loss: 0.4902 - accuracy: 0.7598
Epoch 5/150
964/964 - 2s - loss: 0.4852 - accuracy: 0.7651
Epoch 6/150
964/964 - 1s - loss: 0.4823 - accuracy: 0.7676
Epoch 7/150
964/964 - 1s - loss: 0.4802 - accuracy: 0.7686
Epoch 8/150
964/964 - 1s - loss: 0.4786 - accuracy: 0.7693
Epoch 9/150
964/964 - 1s - loss: 0.4773 - accuracy: 0.7713
Epoch 10/150
964/964 - 1s - loss: 0.4767 - accuracy: 0.7721
Epoch 11/150
964/964 - 1s - loss: 0.4758 - accuracy: 0.7722
Epoch 12/150
964/964 - 1s - loss: 0.4752 - accuracy: 0.7720
Epoch 13/150
964/964 - 1s - loss: 0.4748 - accuracy: 0.7719
Epoch 14/150
964/964 - 1s - loss: 0.4742 - accuracy: 0.7734
Epoch 15/150
964/964 - 1s - loss: 0.4739 - accuracy: 0.7721
Epoch 16/150
964/964 - 1s - loss: 0.4731 - accuracy: 0.7717
Epoch 17/150
964/964 - 1s - loss: 0.4727 - accura

## Validation of the Model

In [14]:
#Evaluate the Model using the testing data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
    
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

322/322 - 0s - loss: 0.4670 - accuracy: 0.7788
Loss: 0.46696773171424866, Accuracy: 0.7788265347480774


## Saving the Trained Model

In [15]:
# # Save the model
model.save("spotify_model.h5")